In [1]:
import numpy as np  # this module handles arrays, but here we need it for its NaN value
import pandas as pd # this module contains a lot of tools for handling tabular data
import re
import gsw

In [2]:
# define paths to the source files and eventual output file
pathBottle='/ocean/eolson/MEOPAR/obs/NemcekHPLC/All 2017 SoG bottle.xlsx'
pathPhyto='/ocean/eolson/MEOPAR/obs/NemcekHPLC/2015-2018 Abs phyto groupsCorrected.xlsx'

pathOut='/ocean/eolson/MEOPAR/obs/NemcekHPLC/bottlePhytoMerged2017.csv'

In [3]:
#formatting function to get year-(3digit) format
def fmtCruise(istr):
    if re.fullmatch('[0-9]{4}-[0-9]{2}',istr): 
        sp=re.split('-',istr)
        rstr=sp[0]+'-0'+sp[1]
    elif re.fullmatch('[0-9]{4}-[0-9]{3}',istr):
        rstr=istr
    else:
        raise ValueError('Input had unexpected format:',istr)
    return rstr

In [4]:
# get names of sheets in notebook
with pd.ExcelFile(pathBottle) as xl:
    sheets=xl.sheet_names
print(sheets)

['2017-01', '2017-63', '2017-05', '2017-64', '2017-09', '2017-65']


In [5]:
# load each sheet in the bottle Excel file and concatenate them together into one table
dfbotlist=list()
for sheet in sheets:
    df0=pd.read_excel(pathBottle,sheet_name=sheet,verbose=True,
                      na_values=(-99,-99.9)) # read each sheet; include additional na values
    df0['Cruise']=fmtCruise(sheet)  # create and populate Cruise column based on sheet name
    dfbotlist.append(df0) # append the sheet to a list
dfbot=pd.concat(dfbotlist,ignore_index=True,sort=False) # concatenate the list into a single table
# Drop columns with no data in them
l1=set(dfbot.keys())
dfbot.dropna(axis=1,how='all',inplace=True)
print('removed empty columns:',l1-set(dfbot.keys()))

Reading sheet 2017-01
Reading sheet 2017-63
Reading sheet 2017-05
Reading sheet 2017-64
Reading sheet 2017-09
Reading sheet 2017-65
removed empty columns: set()


In [6]:
# list the column names in the resulting table
print(dfbot.keys())

Index(['FIL:START TIME YYYY/MM/DD HH:MM:SS', 'LOC:EVENT_NUMBER',
       'LOC:LATITUDE', 'LOC:LONGITUDE', 'LOC:WATER DEPTH', 'ADM:SCIENTIST',
       'LOC:STATION', 'ADM:PROJECT', 'Bottle_Number', 'Sample_Number',
       'Pressure [decibar]', 'Depth [metres]',
       'Temperature:Primary [deg C (ITS90)]',
       'Fluorescence:URU:Seapoint [mg/m^3]', 'PAR [uE/m^2/sec]',
       'Salinity:T0:C0 [PSS-78]', 'Temperature:Draw [deg C (ITS90)]',
       'Salinity:Bottle [PSS-78]', 'Flag:Salinity:Bottle',
       'Chlorophyll:Extracted [mg/m^3]', 'Flag:Chlorophyll:Extracted',
       'Phaeo-Pigment:Extracted [mg/m^3]', 'Oxygen:Dissolved [mL/L]',
       'Oxygen:Dissolved [umol/kg]', 'Flag:Oxygen:Dissolved',
       'Nitrate_plus_Nitrite [umol/L]', 'Flag:Nitrate_plus_Nitrite',
       'Silicate [umol/L]', 'Flag:Silicate', 'Phosphate [umol/L]',
       'Flag:Phosphate', 'Comments by sample_numbeR', 'Cruise', 'Zone',
       'ADM:MISSION', 'Bottle:Firing_Sequence', 'Temperature*',
       'Transmissivity [*/

In [7]:
# no rows returned, so there are no rows with multiple temperature values
print(np.sum(dfbot['Temperature:Primary [deg C (ITS90)]']>=0),
      np.sum(dfbot['Temperature:Secondary [deg C (ITS90)]']>=0),
      np.sum(dfbot['Temperature*']>=0))
dfbot.loc[(np.array([int(ii) for ii in (dfbot['Temperature:Primary [deg C (ITS90)]']>=0)])+\
          np.array([int(ii) for ii in (dfbot['Temperature:Secondary [deg C (ITS90)]']>=0)])+\
           np.array([int(ii) for ii in (dfbot['Temperature*']>=0)]))>1]

204 71 1061


,FIL:START TIME YYYY/MM/DD HH:MM:SS,LOC:EVENT_NUMBER,LOC:LATITUDE,LOC:LONGITUDE,LOC:WATER DEPTH,ADM:SCIENTIST,LOC:STATION,ADM:PROJECT,Bottle_Number,Sample_Number,...,Zone,ADM:MISSION,Bottle:Firing_Sequence,Temperature*,Transmissivity [*/metre],Oxygen:Dissolved:SBE [mL/L],Oxygen:Dissolved:SBE [umol/kg],Temperature:Secondary [deg C (ITS90)],Salinity:T1:C1 [PSS-78],Number_of_bin_records


In [8]:
# no rows returned, so there are no rows with both both salinity fields
print(np.sum(dfbot['Salinity:T0:C0 [PSS-78]']>=0),
      np.sum(dfbot['Salinity:T1:C1 [PSS-78]']>=0))
dfbot.loc[(dfbot['Salinity:T0:C0 [PSS-78]']>=0)&\
          (dfbot['Salinity:T1:C1 [PSS-78]']>=0)]

931 71


,FIL:START TIME YYYY/MM/DD HH:MM:SS,LOC:EVENT_NUMBER,LOC:LATITUDE,LOC:LONGITUDE,LOC:WATER DEPTH,ADM:SCIENTIST,LOC:STATION,ADM:PROJECT,Bottle_Number,Sample_Number,...,Zone,ADM:MISSION,Bottle:Firing_Sequence,Temperature*,Transmissivity [*/metre],Oxygen:Dissolved:SBE [mL/L],Oxygen:Dissolved:SBE [umol/kg],Temperature:Secondary [deg C (ITS90)],Salinity:T1:C1 [PSS-78],Number_of_bin_records


In [9]:
def subval(idf,colList):
    # first value in colList should be the column you are going to keep
    # follow with other columns that will be used to fill in when that column is NaN
    # in order of precedence
    if len(colList)==2:
        idf[colList[0]]=[r[colList[0]] if not pd.isna(r[colList[0]]) \
                         else  r[colList[1]] for i,r in idf.iterrows()]
    elif len(colList)==3:
        idf[colList[0]]=[r[colList[0]] if not pd.isna(r[colList[0]]) \
                         else  r[colList[1]] if not pd.isna(r[colList[1]]) \
                         else r[colList[2]] for i,r in idf.iterrows()]
    else:
        raise NotImplementedError('Add to code to handle this case')
    return idf

In [10]:
# there are some duplicate columns here; handle them:
dfbot=subval(dfbot,('Temperature:Primary [deg C (ITS90)]',
                    'Temperature:Secondary [deg C (ITS90)]',
                    'Temperature*'))
dfbot=subval(dfbot,('Salinity:T0:C0 [PSS-78]',
                    'Salinity:T1:C1 [PSS-78]'))
dfbot.rename(columns={'Temperature:Primary [deg C (ITS90)]':'Temperature [deg C (ITS90)]'},
             inplace=True)
dfbot.rename(columns={'Salinity:T0:C0 [PSS-78]':'Salinity [PSS-78]'},
             inplace=True)

In [11]:
# drop repetetive/unecessary columns:
dfbot.drop(labels=['Temperature:Secondary [deg C (ITS90)]',
                   'Salinity:T1:C1 [PSS-78]','Temperature*'],axis=1,inplace=True)

In [12]:
# define a function that will be applied to the values in the index column;
# this makes it easier to drop non-data rows later
def convertIndex(val):
    try:
        x =int(val)
    except ValueError:
        x=np.nan
    return x

In [13]:
# load the 2015 phytoplankton data with the following options:
#   sheet_name='2015 CHEMTAX abs results'  -> choose the 2015 sheet
#   usecols='A:I,T:AC'   -> read only columns A:I and T:AC from the Excel sheet
#   skiprows=2     -> start reading at the 3rd row of the sheet, 
#                     which contains the column headings
#   converters={'Index': convertIndex,}   -> apply the function defined above to the Index column
#   verbose = True   -> print extra information/ warnings/ errors
dfPhyto=pd.read_excel(pathPhyto,sheet_name='2017 CHEMTAX abs results',usecols='A:I,T:AC',
                      skiprows=2,converters={'Index': convertIndex,},
                      verbose=True)

Reading sheet 2017 CHEMTAX abs results


In [14]:
# display rows 48 to 59 of the resulting table
dfPhyto[48:60]

,Bin #,Index,Subgroup,Cruise,Month,Station,Sample#,rep,depth,Diatoms-1.1,Diatoms-2.1,Prasinophytes.1,Cryptophytes.1,Dinoflagellates-1.1,Haptophytes.1,Dictyo.1,Raphido.1,Cyanobacteria.1,TchlA.1
48,8,44.0,2,2017-63,April,62,103,B,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,8,81.0,2,2017-63,April,59,333,A,0,1.87191,0.0122092,0.000863607,0.121957,0.00183954,0.0789949,0.00239227,0.00977295,0.00176363,2.1017
50,8,82.0,2,2017-63,April,59,333,B,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,8,45.0,2,2017-63,April,56,119,A,0,0.211704,0.0469358,0.000451824,0.0472825,0.000592002,0.00206885,0.000238876,0.00393237,0.000882363,0.314088
52,8,46.0,2,2017-63,April,56,119,B,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,8,47.0,2,2017-63,April,46,132,A,0,0.487764,0.0639207,0.000134122,0.105643,0.000285246,0,5.23371e-05,0.00102591,0.000220098,0.659045
54,8,48.0,2,2017-63,April,46,132,B,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,8,49.0,2,2017-63,April,42,149,A,0,10.0165,0,0,0,0,0,0,0,0,10.0165
56,8,50.0,2,2017-63,April,42,149,B,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,8,51.0,2,2017-63,April,39,167,A,0,9.48799,0,0,0.0628061,0.0263615,0,0,0,0,9.57716


In [15]:
# now, drop any rows from the table that have NaN values in either of the columns
#  'Index' or 'TchlA (ug/L)'
# This is why we applied a function to the Index column to make sure all 
#  non-numeric Index values would have a consistent NaN entry, making them easy to identify
#  and remove
dfPhyto.dropna(subset=['Index', 'TchlA.1'],how='any',inplace=True)

In [16]:
# pandas creates its own index, and after dropping rows I like to reset it -
# this is just for convenience
dfPhyto.reset_index(drop=True,inplace=True)

In [17]:
# apply formatting function all rows in Cruise column to get year-3digit format
dfPhyto['Cruise']=[fmtCruise(ii) for ii in dfPhyto['Cruise']]

In [18]:
# display part of the table, confirming that non-data rows have been removed
dfPhyto[48:60]

,Bin #,Index,Subgroup,Cruise,Month,Station,Sample#,rep,depth,Diatoms-1.1,Diatoms-2.1,Prasinophytes.1,Cryptophytes.1,Dinoflagellates-1.1,Haptophytes.1,Dictyo.1,Raphido.1,Cyanobacteria.1,TchlA.1
48,10,95.0,3,2017-005,May,28,9416,A,0,2.01515,0.0336698,0.0304572,0.224812,0.0588766,0.0514546,0.0124801,0,0,2.4269
49,10,97.0,3,2017-005,May,GEO1,425,A,0,3.0919,0.0550708,0.0019028,0.185077,0.00332519,0.00798773,0.00120004,0.0102796,0.00198157,3.35873
50,10,99.0,3,2017-005,May,38,428,A,0,2.42634,0,0.15152,1.29404,0.354306,0.0345057,0.0298439,0.0253625,0,4.31592
51,10,101.0,3,2017-005,May,41,441,A,0,0.253138,0,0.00143447,0.144226,0.0527428,0,0.000608973,0.00430679,0.0261384,0.482596
52,9,103.0,4,2017-064,June,SI,9,A,0,0.192215,0,0.0965671,0.339983,0.812591,0.528324,0.104858,0.10078,0,2.17532
53,9,105.0,4,2017-064,June,59,23,A,0,0.238003,0.0403578,0.0237206,0.116868,0.0654717,0,0.00691184,0,0,0.491334
54,9,107.0,4,2017-064,June,102,39,A,0,0.648175,0.00135856,0.256468,0.421933,0.475836,0.362364,0.0499411,0.111084,0.0162385,2.3434
55,9,109.0,4,2017-064,June,75,54,A,0,0.808216,0.0611285,0.15788,0.526727,0.534122,0.449798,0.0652406,0.107607,0.019913,2.73063
56,9,111.0,4,2017-064,June,72,68,A,0,0.0532313,0,0.193522,0.538183,0.190127,0.0551871,0.0102039,0.186662,0.000910846,1.22803
57,9,113.0,4,2017-064,June,69,81,A,0,0.0731382,0.0725348,0.176748,0.36403,0.0977205,0.0718542,0.0173697,0.0800267,0,0.953423


In [19]:
# due to repeated column names in the original spreadsheet, '.1' was appended to the names
# of the phytoplankton columns; 
# these lines correct the column names, removing the '.1':
renameDict=dict()
for colName in dfPhyto.keys():
    if colName.endswith('.1'):
        renameDict[colName]=colName.split('.1')[0]
dfPhyto.rename(columns=renameDict,inplace=True)

In [20]:
dfPhyto

,Bin #,Index,Subgroup,Cruise,Month,Station,Sample#,rep,depth,Diatoms-1,Diatoms-2,Prasinophytes,Cryptophytes,Dinoflagellates-1,Haptophytes,Dictyo,Raphido,Cyanobacteria,TchlA
0,14,1.0,1,2017-001,Feb,102,615,A,0,0.317232,0,0.125357,0.172561,0.05328,0.103726,0.00158385,0,0,0.77374
1,14,3.0,1,2017-001,Feb,JF2,627,A,0,0.191759,0,0.183017,0.224788,0.0746315,0.138658,0.00794829,0.0849486,0,0.905752
2,14,5.0,1,2017-001,Feb,59,641,A,0,0.162127,0.226224,0.0844669,0.106139,0.000136995,0,0.000825349,0,0,0.579919
3,14,7.0,1,2017-001,Feb,56,655,A,0,0.400149,0.339986,0.185261,0.23638,0.0544237,0,0.0213659,0.0652725,0.00652189,1.30936
4,14,9.0,1,2017-001,Feb,46,667,A,0,0.400165,0.343946,0.154102,0.196001,0.0476347,0,0.0230265,0.0183578,0.00952713,1.19276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,9,221.0,6,2017-065,Oct,9,260,A,0,0.00139186,0,0.06553,0.0532288,7.60988e-05,0.482544,0.00228389,0,0.0332161,0.638271
113,9,223.0,6,2017-065,Oct,12,276,A,0,1.39607,0.304611,0.00015934,0.0995238,0.00121848,0.118025,0.0128329,0.00394914,0.000149612,1.93654
114,9,225.0,6,2017-065,Oct,14,291,A,0,0.0332897,0,0.123427,0.0855781,0.000835815,0.363145,0.00239296,0.114268,0.0236563,0.746593
115,9,227.0,6,2017-065,Oct,16,304,A,0,0.434865,0.0029393,0.00921401,0.0648194,0.000819446,0.0193703,0.00647362,0.00496289,0.000135513,0.543599


In [21]:
# This is the important step- join the two tables ('left' and 'right'), 
#  matching the cruise IDs and sample numbers
#   how='outer'  -> all rows from both the left and the right tables will be included, 
#                   even if they cannot be matched; this makes it easy to check for 
#                   unmatched data later
#   left_on  specifies the name of the column to match in the left table (dfbot) 
#   right_on specifies the name of the column to match in the right table (dfPhyto)
dfout = pd.merge(dfbot, dfPhyto,  how='outer', 
                 left_on=['Cruise','Sample_Number'], right_on = ['Cruise','Sample#'])

In [22]:
# show the column names in the resulting table
dfout.keys()

Index(['FIL:START TIME YYYY/MM/DD HH:MM:SS', 'LOC:EVENT_NUMBER',
       'LOC:LATITUDE', 'LOC:LONGITUDE', 'LOC:WATER DEPTH', 'ADM:SCIENTIST',
       'LOC:STATION', 'ADM:PROJECT', 'Bottle_Number', 'Sample_Number',
       'Pressure [decibar]', 'Depth [metres]', 'Temperature [deg C (ITS90)]',
       'Fluorescence:URU:Seapoint [mg/m^3]', 'PAR [uE/m^2/sec]',
       'Salinity [PSS-78]', 'Temperature:Draw [deg C (ITS90)]',
       'Salinity:Bottle [PSS-78]', 'Flag:Salinity:Bottle',
       'Chlorophyll:Extracted [mg/m^3]', 'Flag:Chlorophyll:Extracted',
       'Phaeo-Pigment:Extracted [mg/m^3]', 'Oxygen:Dissolved [mL/L]',
       'Oxygen:Dissolved [umol/kg]', 'Flag:Oxygen:Dissolved',
       'Nitrate_plus_Nitrite [umol/L]', 'Flag:Nitrate_plus_Nitrite',
       'Silicate [umol/L]', 'Flag:Silicate', 'Phosphate [umol/L]',
       'Flag:Phosphate', 'Comments by sample_numbeR', 'Cruise', 'Zone',
       'ADM:MISSION', 'Bottle:Firing_Sequence', 'Transmissivity [*/metre]',
       'Oxygen:Dissolved:SBE [mL/L]

### Checks

In [23]:
# Identify cases where phytoplankton data were matched to multiple samples in bottle data:
dftest=pd.merge(dfbot, dfPhyto,how='right', left_on=['Cruise','Sample_Number'],right_on = ['Cruise','Sample#'])


In [24]:
temp=dftest.groupby(['Cruise','Sample#']).agg({'Cruise':['count']})
temp.columns = ['icount']

In [25]:
np.unique(temp.icount)

array([1])

In [26]:
# check for Phyto samples matched to multiple bottle samples:
temp.loc[temp.icount>1]

,,icount
Cruise,Sample#,


In [27]:
# check for phyto samples not matched to bottle samples:
temp.loc[temp.icount==0]

,,icount
Cruise,Sample#,


In [28]:
temp2=dfout.groupby(['Cruise','Sample_Number']).agg({'Cruise':['count']})
temp2.columns = ['icount']
# this will catch phyto matched to multiple bottle but also bottle with duplicate sample numbers per cruise:
temp2.loc[temp2.icount>1]

,,icount
Cruise,Sample_Number,
2017-064,-99,7


In [29]:
# check for phyto samples not matched to bottle samples:
temp.loc[temp.icount==0]

,,icount
Cruise,Sample#,


In [30]:
# if the output table is longer than either of the input tables, some columns were not matched
len(dfout), len(dfPhyto), len(dfbot)

(1336, 117, 1336)

In [31]:
# Check that the number of cells with data in the 'Cyanobacteria' column is 
#  the same for the input and output tables to show that no rows are missing:
np.sum(dfPhyto['Cyanobacteria']>=0), np.sum(dfout['Cyanobacteria']>=0)

(117, 117)

In [32]:
# If there were data rows from the phytoplankton table that were not matched to 
#  rows from the bottle table, their indices from the phytoplankton table would be 
#  displayed below (the series [] would not be empty)
print(dfout.loc[dfout['ADM:SCIENTIST'].isna()]['Index'])

Series([], Name: Index, dtype: float64)


In [33]:
# drop repetetive/unecessary columns:
dfout.drop(labels=['Bin #', 'Index', 'Subgroup', 'Month', 'Station', 'Sample#', 'rep',
                   'depth',],axis=1,inplace=True)

In [34]:
# truncate phyto group values to 3 decimal places:
for col in ('Cyanobacteria', 'Prasinophytes', 'Cryptophytes', 'Diatoms-1',
       'Diatoms-2', 'Dinoflagellates-1', 'Haptophytes', 'Dictyo', 'Raphido',
       'TchlA'):
    dfout[col]=[np.round(ii,decimals=3) for ii in dfout[col]] # use list comprehension to set values for entire column

In [35]:
dfout['Cyanobacteria']

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
        ...  
1331      NaN
1332    0.000
1333      NaN
1334      NaN
1335    0.043
Name: Cyanobacteria, Length: 1336, dtype: float64

In [36]:
# now write the output table to a .csv file:
dfout.to_csv(pathOut, index=False)  

In [37]:
dfout.keys()

Index(['FIL:START TIME YYYY/MM/DD HH:MM:SS', 'LOC:EVENT_NUMBER',
       'LOC:LATITUDE', 'LOC:LONGITUDE', 'LOC:WATER DEPTH', 'ADM:SCIENTIST',
       'LOC:STATION', 'ADM:PROJECT', 'Bottle_Number', 'Sample_Number',
       'Pressure [decibar]', 'Depth [metres]', 'Temperature [deg C (ITS90)]',
       'Fluorescence:URU:Seapoint [mg/m^3]', 'PAR [uE/m^2/sec]',
       'Salinity [PSS-78]', 'Temperature:Draw [deg C (ITS90)]',
       'Salinity:Bottle [PSS-78]', 'Flag:Salinity:Bottle',
       'Chlorophyll:Extracted [mg/m^3]', 'Flag:Chlorophyll:Extracted',
       'Phaeo-Pigment:Extracted [mg/m^3]', 'Oxygen:Dissolved [mL/L]',
       'Oxygen:Dissolved [umol/kg]', 'Flag:Oxygen:Dissolved',
       'Nitrate_plus_Nitrite [umol/L]', 'Flag:Nitrate_plus_Nitrite',
       'Silicate [umol/L]', 'Flag:Silicate', 'Phosphate [umol/L]',
       'Flag:Phosphate', 'Comments by sample_numbeR', 'Cruise', 'Zone',
       'ADM:MISSION', 'Bottle:Firing_Sequence', 'Transmissivity [*/metre]',
       'Oxygen:Dissolved:SBE [mL/L]